In [1]:
!pip install tensorflow==1.14
import matplotlib.pyplot as plt
import numpy as np
import os
%tensorflow_version 1.x
import tensorflow as tf
import time

from tqdm import trange
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

LOG_DIR = './logs'

     |████████████████████████████████| 109.2MB 99kB/s 
     |████████████████████████████████| 491kB 43.7MB/s 
     |████████████████████████████████| 3.2MB 52.4MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
TensorFlow 1.x selected.
Found GPU at: /device:GPU:0


In [9]:
def build_better_linear_model(model_name, learning_rate, batch_size, reuse=False, **kwargs): 
  def _linear(input_tensor, input_dim, output_dim, layer_name):
    with tf.variable_scope(model_name + layer_name, reuse=tf.compat.v1.AUTO_REUSE):
      with tf.variable_scope("weights"):
        W = tf.get_variable("W", shape=(input_dim, output_dim))
      with tf.variable_scope("biases"):
        b = tf.get_variable("b", shape=(output_dim))
      return tf.matmul(input_tensor, W) + b
    
  def _linear_with_normal(input_tensor, input_dim, output_dim, layer_name):  
    with tf.variable_scope(model_name + layer_name, reuse=tf.compat.v1.AUTO_REUSE):
      with tf.variable_scope("weights"):
        W = tf.get_variable("W", shape=(input_dim, output_dim))
      with tf.variable_scope("gamma"):
        gam = tf.get_variable("gamma", shape=(output_dim))
      with tf.variable_scope("beta"):
        beta = tf.get_variable("beta", shape=(output_dim))
      return tf.matmul(input_tensor, W) * (gam+1) + beta
       
  class Model(object):
    pass
  
  model = Model()
  img_size = kwargs["image_size"]
  
  images = 1
  
  if len(img_size) == 2:
    images = tf.placeholder(shape=(None, None, None), dtype=tf.float32, name="images")
  else:
    images = tf.placeholder(shape=(None, None, None, None), dtype=tf.float32, name="images")

  labels = tf.placeholder(shape=(None), dtype=tf.int32, name="labels")
  
  model.inputs = [images, labels]
  
  train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
  train_dataset = train_dataset.shuffle(1000)
  train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
  
  train_dataset = train_dataset.repeat()
  
  test_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
  test_dataset = test_dataset.batch(batch_size, drop_remainder = True)
  
  model.train_iterator = train_dataset.make_initializable_iterator()
  model.test_iterator = test_dataset.make_initializable_iterator()
  
  model.handle = tf.placeholder(tf.string, shape=[])
  model.iterator = tf.data.Iterator.from_string_handle(
      model.handle, train_dataset.output_types, train_dataset.output_shapes)
  
  images, labels = model.iterator.get_next()
  

  # Flatten image
  x = tf.reshape(images, [-1,  kwargs["pixels"]])
  sizes = kwargs["sizes"]
  last_size = kwargs["pixels"]
  for id, size in enumerate(sizes):
    x = _linear_with_normal(input_tensor=x, 
                     input_dim=last_size, 
                     output_dim=size , 
                     layer_name='linear1'+str(id))

    x = tf.nn.relu(x)
    if reuse==False:
      x = tf.nn.dropout(x, keep_prob=(1-kwargs["dropout"]))
    last_size = size
  
  logits = _linear(input_tensor = x, 
                  input_dim = last_size,
                  output_dim = 10,
                  layer_name = 'linear21')

  model.probs = tf.nn.softmax(logits)
  
  # Compute loss
  with tf.variable_scope("loss"):
    model.loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, 
                                                        logits=logits)
  
  # Compute accuracy
  with tf.variable_scope("accuracy"):
    model.correct_predictions = tf.equal(
        labels, tf.reshape(tf.argmax(model.probs, axis=1, output_type=tf.int32), [-1, 1]))
    accuracy = tf.reduce_mean(tf.cast(model.correct_predictions, tf.float32))

  # Optimize
  model.train_op = tf.train.GradientDescentOptimizer(
      learning_rate=learning_rate).minimize(model.loss)
 
  return model

In [3]:
def evaluate_better_linear_model(sess, model, test_images, test_labels):
  images, labels = model.inputs
  sess.run(model.test_iterator.initializer, feed_dict={
      images: test_images,
      labels: test_labels
  })
  test_handle = sess.run(model.test_iterator.string_handle())
  
  correct_predictions = 0.0
  total_predictions = 0
  while True:
    try:
      predictions = sess.run(model.correct_predictions,
                             feed_dict={model.handle: test_handle})
      correct_predictions += np.sum(predictions)
      total_predictions += predictions.shape[0]
    except tf.errors.OutOfRangeError:
      break
  
  return correct_predictions / total_predictions

In [7]:
def _create_nn(model_name, train_images, train_labels, test_images, test_labels, **params):
  tf.reset_default_graph()

  model = build_better_linear_model(model_name = model_name, **params)
  eval_model = build_better_linear_model(model_name = model_name, reuse=True, **params)

  timestamp = int(time.time())
  writer = tf.summary.FileWriter(
      os.path.join(LOG_DIR, 'better_linear_model_%d' % timestamp), 
      graph = tf.get_default_graph())

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    train_handle = sess.run(model.train_iterator.string_handle()) #zwroc iterator do danych treningowych
    
    images, labels = model.inputs
    sess.run(model.train_iterator.initializer, feed_dict={
        images: train_images,
        labels: train_labels
    })

    steps_per_epoch = train_images.shape[0] // params['batch_size'] 

    for epoch in range(params['num_epochs']):
      for i in range(steps_per_epoch):
        step = epoch * steps_per_epoch + i
        sess.run(model.train_op,
                   feed_dict={model.handle: train_handle})

      print("Epoch %d: %.4f" % (
          epoch, evaluate_better_linear_model(sess, eval_model, test_images, test_labels)))

  writer.close()

In [ ]:
params = {
    'batch_size': 64, 
    'num_epochs': 50,
    'learning_rate': 0.1,
    'log_summaries_every': 10000,
    'dropout': 0.1,
    'sizes': [2000, 1000, 500],
    'image_size': (32, 32, 3),
    'pixels': 3072
}

cifar10 = tf.keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = np.array(train_images, dtype=np.float32) / 255
test_images = np.array(test_images, dtype=np.float32) / 255

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

_create_nn("cifar", train_images, train_labels, test_images, test_labels, **params)